# Main notebook for data processing in robo_romeo project

## Imports - this should do us for the whole project. 

In [1]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception #to get pre-trained model Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer #for text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense#Keras to build our CNN and LSTM
from tensorflow.keras.layers import LSTM, Embedding, Dropout
from tqdm import tqdm_notebook as tqdm #to check loop progress


/var/folders/tw/_zhlxthd7c31pn2_v_8xnv3m0000gn/T/ipykernel_25854/3341886489.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

## Data cleaning

 - load_doc( filename ) – To load the document file and read the contents of the file into a string.

In [2]:
# Load the document file into memory
def load_doc(filename):
    # Open file to read
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [4]:
filename = '../raw_data/text/Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)

 - load_descriptions(doc) – To create a description dictionary that will map images with all 5 captions.

In [5]:
# extract descriptions for images
def load_descriptions(doc):
    mapping = dict()
    # process lines
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        if len(line) < 2:
            continue
        # take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
        # remove filename from image id
        image_id = image_id.split('.')[0]
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping

In [6]:
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


 - clean_descriptions( descriptions) – to clean the data by taking all descriptions as input. This will perform several types of cleaning including uppercase to lowercase conversion, punctuation removal, and removal of the number containing words.



In [7]:
import string

In [8]:
def clean_descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # store as string
            desc_list[i] =  ' '.join(desc)

In [9]:
# clean descriptions
clean_descriptions(descriptions)

 - txt_vocab( descriptions ) – to create a vocabulary from all the unique words extracted out from descriptions.



In [10]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc

In [11]:
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 8763


 - save_descriptions( descriptions, filename ) – This function is used to store all the preprocessed descriptions into a file.



In [12]:
!pwd

/Users/theebak/code/theebak31/robo_romeo/notebooks


In [13]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
         

In [14]:
# save descriptions
save_descriptions(descriptions, 'descriptions.txt')

In [17]:
!pwd

/Users/theebak/code/theebak31/robo_romeo/notebooks


In [18]:
filename = '../raw_data/text/Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, 'descriptions.txt')


Loaded: 8092 
Vocabulary Size: 8763


In [73]:
def sequencing(file):
    
    with open(file) as f:
        lines = f.readlines()
    #splitting the lines in to lists
    
    list = []
    training_list = []
    X1 = []
    X2 = []
    y = []
    token_vocab = []
    
    k_file = "../raw_data/text/Flickr_8k.trainImages.txt"

    with open(k_file) as n:
        l = n.readlines()
    for line in l:
        training_list.append(line.replace(".jpg\n", ""))
    
    for j in lines:
        list.append(j.split())
    
    for item in list:
        item.insert(1,"startsequence")
        item.append("endsequence")
        token_vocab.append(' '.join(item[1:]))
        
    
    # for loop to append X1,X2,y

    
    for i in list:
        for seq in range(2,len(i)):
            if i[0] in training_list:
                X1.append(i[0])
                X2.append(' '.join(i[1:seq]))
                y.append(i[seq])
            else:
                pass


    return(X1,X2,y,token_vocab)

In [100]:
X1,X2,y,token_vocab = sequencing('descriptions.txt')

In [75]:
len(set(y))

7577

In [101]:
token_vocab[0]

['startsequence',
 'a',
 'child',
 'in',
 'a',
 'pink',
 'dress',
 'is',
 'climbing',
 'up',
 'a',
 'set',
 'of',
 'stairs',
 'in',
 'an',
 'entry',
 'way',
 'endsequence']

In [102]:
#Tokenizing X2
t = Tokenizer()
t.fit_on_texts(token_vocab)
X2_tokenized = t.texts_to_sequences(X2)

#Tokenizing y
y_tokenized = np.array(t.texts_to_sequences(y))

In [103]:
y_tokenized[14]

array([28])

In [104]:
y_vocab = len(t.word_index.keys())
y_vocab

8777

In [105]:
y_tokenized.shape

(353505, 1)

In [106]:
y = to_categorical(y_tokenized, num_classes=y_vocab+1)

In [107]:
X2_tokenized

[[2],
 [2, 1],
 [2, 1, 42],
 [2, 1, 42, 4],
 [2, 1, 42, 4, 1],
 [2, 1, 42, 4, 1, 90],
 [2, 1, 42, 4, 1, 90, 170],
 [2, 1, 42, 4, 1, 90, 170, 7],
 [2, 1, 42, 4, 1, 90, 170, 7, 119],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396, 12],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396, 12, 393],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396, 12, 393, 4],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396, 12, 393, 4, 28],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396, 12, 393, 4, 28, 5199],
 [2, 1, 42, 4, 1, 90, 170, 7, 119, 53, 1, 396, 12, 393, 4, 28, 5199, 693],
 [2],
 [2, 1],
 [2, 1, 19],
 [2, 1, 19, 314],
 [2, 1, 19, 314, 64],
 [2, 1, 19, 314, 64, 1],
 [2, 1, 19, 314, 64, 1, 194],
 [2, 1, 19, 314, 64, 1, 194, 117],
 [2],
 [2, 1],
 [2, 1, 40],
 [2, 1, 40, 19],
 [2, 1, 40, 19, 119],
 [2, 1, 40, 19, 119, 64],
 [2, 1, 40, 19, 119, 64, 1],
 [2, 1, 40, 19, 119, 64, 1, 19

In [108]:
#padding
X2_pad = pad_sequences(X2_tokenized, dtype='int32', padding='post', value=0)

In [109]:
f = X1,X2_pad,y

In [110]:
y.shape

(353505, 8778)

In [111]:
t.index_word

{1: 'a',
 2: 'startsequence',
 3: 'endsequence',
 4: 'in',
 5: 'the',
 6: 'on',
 7: 'is',
 8: 'and',
 9: 'dog',
 10: 'with',
 11: 'man',
 12: 'of',
 13: 'two',
 14: 'white',
 15: 'black',
 16: 'boy',
 17: 'are',
 18: 'woman',
 19: 'girl',
 20: 'to',
 21: 'wearing',
 22: 'at',
 23: 'people',
 24: 'water',
 25: 'red',
 26: 'young',
 27: 'brown',
 28: 'an',
 29: 'his',
 30: 'blue',
 31: 'dogs',
 32: 'running',
 33: 'through',
 34: 'playing',
 35: 'while',
 36: 'down',
 37: 'shirt',
 38: 'standing',
 39: 'ball',
 40: 'little',
 41: 'grass',
 42: 'child',
 43: 'person',
 44: 'snow',
 45: 'jumping',
 46: 'over',
 47: 'front',
 48: 'three',
 49: 'sitting',
 50: 'holding',
 51: 'field',
 52: 'small',
 53: 'up',
 54: 'by',
 55: 'large',
 56: 'green',
 57: 'one',
 58: 'group',
 59: 'yellow',
 60: 'her',
 61: 'walking',
 62: 'children',
 63: 'men',
 64: 'into',
 65: 'air',
 66: 'beach',
 67: 'near',
 68: 'mouth',
 69: 'jumps',
 70: 'another',
 71: 'for',
 72: 'street',
 73: 'runs',
 74: 'its',
 7

In [112]:
#Saving Caption file
file = "../raw_data/captions/cap"
outfile = open(file,'wb')
dump(f,outfile)
outfile.close()

In [53]:
file = "../raw_data/captions/cap"

In [54]:
with open(file, 'rb') as handle:
    b = load(handle)